## Deep Learning Study
#### Week 7: Language Models(BERT & GPT)
![image](https://github.com/HanyangTechAI/2023-Deep-Learning-Study/assets/44901828/cfb43445-7e90-42e4-a442-653fe08c36c5)
- BERT는 Transformer 아키텍처를 기반으로 한 모델로, 문장의 의미를 잘 이해하고 문장을 벡터 형태로 변환할 수 있습니다.
- Sentence embedding은 문장을 고정된 길이의 벡터로 표현하는 것을 의미하며, 문장 간의 유사도를 계산하는 데 유용합니다. 자세한 내용은 [Weekly NLP week4](https://jiho-ml.com/weekly-nlp-4/)를 참고하세요!
- 이번 과제에서는 BERT 모델을 사용하여 문장을 임베딩하고, cosine similarity와 같은 유사도 계산을 수행하여 문장 간의 관계를 분석합니다.
- BERT를 사용하여 sentence embedding을 계산하는 방법과 cosine similarity를 계산하는 방법을 학습하고, 주어진 문장들에 대해 이를 적용하여 관계를 파악해보세요.

In [1]:
# 현재 시스템에 transformers와 datasets 라이브러리를 설치합니다.
!pip install transformers datasets --quiet
import torch
import numpy as np

# GPU 사용 여부
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

- 입력된 문장 사이의 유사도를 계산하기 위해서는, 각 문장의 특성을 잘 파악해 벡터 공간에 대응시키도록 학습된 모델이 필요합니다.
- 한국어 문장 임베딩을 계산할 수 있도록 공개된 [레포지토리](https://github.com/BM-K/Sentence-Embedding-Is-All-You-Need)에서 소개된 모델과 토크나이저를 불러와 사용해 보겠습니다.

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta-multitask').to(device).eval()  # or 'BM-K/KoSimCSE-bert-multitask'
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta-multitask')  # or 'BM-K/KoSimCSE-bert-multitask'

![image](https://github.com/HanyangTechAI/2023-Deep-Learning-Study/assets/44901828/e988d25c-84c5-4f19-b8a9-dd82ff0bce53)

- 유사도를 계산하기 위해 3개의 예시 문장을 준비해 보았습니다.
- 미리 불러온 토크나이저를 활용해 입력 문장을 토큰 인덱스의 배열로 변환하고, 모델에 입력하여 출력 결과물을 가져옵니다.
- 각각의 문장에 대한 임베딩은 (문장 길이, 모델의 dimension)으로, 문장 사이의 유사도를 계산하기 위해서 0번째 토큰([CLS])에 해당하는 임베딩만 사용합니다.

In [3]:
sentences = ['치타가 들판을 가로 질러 먹이를 쫓는다.',
             '치타 한 마리가 먹이 뒤에서 달리고 있다.',
             '원숭이 한 마리가 드럼을 연주한다.']

# 입력 문장을 tokenizer를 사용하여 토큰 인덱스 배열로 변환합니다.
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
# Train을 진행하지 않고 inference만 하기 때문에,
# gradient를 계산하지 않도록 하여 메모리 사용량을 절약합니다.
with torch.no_grad():
  # 모델의 출력값과 loss를 계산합니다. loss는 사용하지 않습니다.
  embeddings, loss = model(**inputs, return_dict=False) 

for i, emb in enumerate(embeddings):
  print(f"{i}번째 문장: {sentences[i]}")
  print(f"생성된 embedding vector의 shape: {emb.shape}")

0번째 문장: 치타가 들판을 가로 질러 먹이를 쫓는다.
생성된 embedding vector의 shape: torch.Size([15, 768])
1번째 문장: 치타 한 마리가 먹이 뒤에서 달리고 있다.
생성된 embedding vector의 shape: torch.Size([15, 768])
2번째 문장: 원숭이 한 마리가 드럼을 연주한다.
생성된 embedding vector의 shape: torch.Size([15, 768])


![image](https://github.com/HanyangTechAI/2023-Deep-Learning-Study/assets/44901828/19ece8f7-5c7a-43da-add4-d99f50165863)
- 생성된 임베딩의 Cosine similarity를 계산하여 두 문장이 얼마나 의미적으로 유사한지, 즉 임베딩 공간 상에서 얼마나 가까이 있는지 알아보도록 하겠습니다.
- Cosine simiarity란 두 벡터 사이의 관계를 계산할 수 있는 방법 중 하나입니다. 두 벡터 사이의 각도 theta가 있을 때, cosine(theta)의 값이 크면 클 수록 두 벡터가 유사하고, 작을수록 다르다는 것을 의미합니다.
- Cosine similarity에 대한 자세한 내용이 궁금하다면 [Weekly NLP week5](https://jiho-ml.com/weekly-nlp-5/)를 참고하세요!

In [4]:
def cal_score(a, b):
    # 0번째 토큰에 해당하는 임베딩 벡터만 사용
    a = a[0:1]
    b = b[0:1]

    # 각 벡터 정규화
    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]

    # 정규화된 두 벡터를 곱하여 유사도 점수 계산
    return torch.mm(a_norm, b_norm.transpose(0, 1)).item() * 100

- 두 문장의 유사도를 계산하는 ```cal_score```함수의 실행 과정은 아래와 같이 이루어집니다.
  - 입력된 문장은 각각 (문장 길이, 모델의 hidden dimension(768))의 벡터 형태로 변환됩니다.
  - 각 문장의 임베딩 벡터 중 0번째 토큰([CLS])에 해당하는 벡터만 선택합니다. 각각의 벡터는 (1, 768) 형태가 됩니다. 
  - 두 벡터의 Cosine Similarity를 계산한 뒤 100을 곱해 두 문장의 유사도를 %로 나타냅니다.

In [5]:
score01 = cal_score(embeddings[0], embeddings[1])  # 약 84%
# '치타가 들판을 가로 질러 먹이를 쫓는다.' @ '치타 한 마리가 먹이 뒤에서 달리고 있다.'
score02 = cal_score(embeddings[0], embeddings[2])  # 약 23%
# '치타가 들판을 가로 질러 먹이를 쫓는다.' @ '원숭이 한 마리가 드럼을 연주한다.'

print(f"'{sentences[0]}'와 '{sentences[1]}'의 유사도: {format(score01, '.2f')}%")
print(f"'{sentences[0]}'와 '{sentences[2]}'의 유사도: {format(score02, '.2f')}%")

'치타가 들판을 가로 질러 먹이를 쫓는다.'와 '치타 한 마리가 먹이 뒤에서 달리고 있다.'의 유사도: 84.10%
'치타가 들판을 가로 질러 먹이를 쫓는다.'와 '원숭이 한 마리가 드럼을 연주한다.'의 유사도: 23.22%


- Huggingface는 다양한 언어와 task를 사용하는 데이터셋을 쉽게 사용할 수 있도록 하는 **dataset** 라이브러리를 제공합니다.
- Huggingface는 model hub와 마찬가지로 다양한 데이터셋을 찾을 수 있는 [dataset hub](https://huggingface.co/datasets)도 운영하고 있습니다.
- 네이버 뉴스 기사를 저장해둔 텍스트 데이터셋을 불러와서 살펴보도록 하겠습니다.

In [6]:
from datasets import load_dataset

# 네이버 뉴스 요약 데이터셋 다운로드
# 데이터셋 주소
naver_news = load_dataset("daekeun-ml/naver-news-summarization-ko")

  0%|          | 0/3 [00:00<?, ?it/s]

- 다운로드한 데이터셋을 살펴보면, train/validation/test 세 가지의 split으로 구성되어 있는 것을 볼 수 있습니다.
- test split을 선택해보면, 각각의 데이터셋은 table 형태로 구성되어 있고, 날짜, 제목, 본문, 요약문 등 다양한 column이 존재하는 것을 알 수 있습니다.
- 데이터셋에서 뉴스 제목을 500개 정도만 추출하여 사용해 보겠습니다.

In [20]:
print("전체 데이터셋:\n", naver_news)
print("Test split:\n", naver_news["test"])

news_titles = naver_news["test"]["title"][:500]
print("뉴스 제목:")
for title in news_titles[:5]:
  print(title)

전체 데이터셋:
 DatasetDict({
    train: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2466
    })
    test: Dataset({
        features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
        num_rows: 2740
    })
})
Test split:
 Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 2740
})
뉴스 제목:
아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원
뉴스페이스로 가는 한국… 우주기업 성장할 기술·터전 만든다
SP500 올 상반기 21%↓…50년래 최악 월가월부
풀무원 알래스칸 명태살 ‘볼카츠’ 출시
일동제약 생산본부장에 강덕원 부사장 영입


#### 여기부터 과제 시작
----
# 7주차 과제: Sentence embedding을 활용한 뉴스 제목 추천
- 위에서 사용했던 BERT 모델을 이용하여, 사용자가 입력한 내용과 가장 유사한 뉴스 제목을 찾아 추천해주는 프로그램을 만들어 봅시다.
- 프로그램의 뼈대를 이루는 함수 이름과 정의가 마련되어 있으니, 필요한 부분을 자유롭게 구성하여 프로그램을 완성해 봅시다.
- ***주의: main 함수의 정의는 바꾸지 마세요!***

### 작동 예시
```
입력 > 신재생 에너지
출력 > 입력과 유사한 상위 5개의 뉴스 제목을 출력합니다.
1. 에너지정책 유턴  재생에너지 대신 원전으로	(score: 56.85)
2. 전기 아껴쓴 만큼 현금 돌려받는다...에너지캐쉬백 전국 확대	(score: 54.78)
3. 수자원공사 한국에너지기술연과 그린수소 연구 업무협약	(score: 53.07)
4. 이어지는 폭염 늘어나는 전력수요	(score: 51.01)
5. 코오롱 수소 밸류체인 플랫폼 구축… 생산부터 발전까지 원스톱	(score: 50.95)
```

### Hint
- Hint 1: news_title에는 뉴스 데이터셋에 포함되어 있는 각 뉴스 기사들의 제목이 포함되어 있습니다. 예시 문장의 임베딩을 계산한 것과 같은 방식을 사용할 수 있습니다.
- Hint 2: 위에서 정의하고 사용했던 ```cal_score``` 함수를 사용하면 사용자가 입력한 요청사항과 뉴스 제목들 사이의 similarity score를 계산할 수 있습니다.

In [23]:
from tqdm import tqdm
def get_sentence_embeddings(sentences):
  # 입력 문장을 tokenizer를 사용하여 토큰 인덱스 배열로 변환합니다.
  inputs = None
  # 모델을 활용하여 임베딩을 계산합니다..
  with torch.no_grad():
    embeddings, loss = None
  return embeddings

def get_similarity_scores(embeddings, input_text):
  # 입력 텍스트에 대한 임베딩을 계산합니다.
  input_embedding = None

  # 입력 텍스트의 임베딩과 미리 계산된 임베딩 사이의 유사도를 계산합니다.
  scores = []
  for emb in embeddings:
    scores.append(None)

  return scores

In [24]:
# 이 부분은 수정하지 마세요!

def main():
  # 뉴스 기사 제목의 임베딩 벡터 계산
  news_embeddings = get_sentence_embeddings(news_titles)

  input_text = input("입력 > ")
  # 입력된 텍스트와 뉴스 기사 제목들 사이의 유사도 계산
  similarity_scores = get_similarity_scores(news_embeddings, input_text)

  print("출력 > 입력과 유사한 상위 5개의 뉴스 제목을 출력합니다.")
  # 유사도 점수가 가장 높은 5개의 index 찾기
  top_5_idx = reversed(np.argsort(similarity_scores)[-5:])

  # 각 index에 해당하는 뉴스 제목과 유사도 점수 출력
  for i, idx in enumerate(top_5_idx):
    print(f"{i+1}. {news_titles[idx]}\t(score: {format(similarity_scores[idx], '.2f')})")

# 프로그램 실행
main()

입력 > 신재생 에너지
출력 > 입력과 유사한 상위 5개의 뉴스 제목을 출력합니다.
1. 에너지정책 유턴  재생에너지 대신 원전으로	(score: 56.85)
2. 전기 아껴쓴 만큼 현금 돌려받는다...에너지캐쉬백 전국 확대	(score: 54.78)
3. 수자원공사 한국에너지기술연과 그린수소 연구 업무협약	(score: 53.07)
4. 이어지는 폭염 늘어나는 전력수요	(score: 51.01)
5. 코오롱 수소 밸류체인 플랫폼 구축… 생산부터 발전까지 원스톱	(score: 50.95)
